In [11]:
from sklearn import datasets, linear_model, neighbors, svm, ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from base import SuperLearner
from base import BMA
import pandas as pd
import numpy as np
from pyearth import Earth
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # warnings from py-earth

v_folds = 5
ols = linear_model.LinearRegression()
elnet = linear_model.ElasticNetCV(l1_ratio=0.5, cv=v_folds, normalize=True)
ridge = linear_model.RidgeCV(cv=v_folds)
lars = linear_model.LarsCV(cv=v_folds, normalize=True)
lasso = linear_model.LassoCV(cv=v_folds, normalize=True)
nn = neighbors.KNeighborsRegressor(weights='distance')
svm1 = svm.SVR(kernel='linear', C=10, gamma='auto')
svm2 = svm.SVR(kernel='poly', C=10, gamma='auto')
rf = ensemble.RandomForestRegressor(n_estimators=100,min_samples_split=5)
gbm = ensemble.GradientBoostingRegressor()
# earth is sort of like D/S/A?
earth=Earth(max_terms=50,max_degree=3,use_fast=True,verbose=0) # get this from https://github.com/scikit-learn-contrib/py-earth
rtree=DecisionTreeRegressor()

def sim1(n):
    w=np.random.binomial(1,.4,size=(10,n))
    eps=np.random.normal(0,1,size=n)
    y=2*w[0]*w[9]+4*w[1]*w[6]+3*w[3]*w[4]-\
    5*w[5]*w[9]+3*w[7]*w[8]+w[0]*w[1]*w[3]-\
    2*w[6]*(1-w[5])*w[1]*w[8]-4*(1-w[9])*w[0]*(1-w[3])+eps
    return np.transpose(w),y

train1,test1=sim1(500),sim1(10000)

In [12]:
# instantiate the super learner!
SuperLearner(cand_learners=[ols,lars,earth,rf],V=10
            ).debug(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE,Coefs
0,LinearRegression,5.835790,6.099963,6.508636,-0.144438
1,LarsCV,5.845518,6.075530,6.490248,0.119983
2,Earth,1.008786,1.127030,1.213925,0.907129
3,RandomForestRegressor,0.786072,1.995398,1.898001,0.114137
4,Meta (LinearRegression),0.931397,1.115300,1.197665,NaN


In [13]:
SuperLearner(cand_learners=[ols,lars,earth,rf],V=10,meta_learner=lasso
            ).debug(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE,Coefs
0,LinearRegression,5.835790,6.099963,6.508636,-0.025871
1,LarsCV,5.845518,6.075530,6.490248,-0.000000
2,Earth,1.008786,1.127030,1.213925,0.908660
3,RandomForestRegressor,0.781368,2.024548,1.901369,0.109250
4,Meta (LassoCV),0.934021,1.115322,1.198354,NaN


In [14]:
SuperLearner(cand_learners=[ols,lars,earth,rf],V=10,meta_learner=rtree
            ).debug(*train1,*test1)

,Learner,Train MSE,Train CV MSE,Test MSE
0,LinearRegression,5.835790,6.099963,6.508636
1,LarsCV,5.845518,6.075530,6.490248
2,Earth,1.008786,1.127030,1.213925
3,RandomForestRegressor,0.794035,2.039409,1.946362
4,Meta (DecisionTreeRegressor),1.647171,0.029391,2.211951


In [23]:
diabetes=datasets.load_diabetes()

SuperLearner(cand_learners=[ols,lars,earth,rf],V=10,meta_learner=lasso
            ).debug(diabetes.data,diabetes.target)

,Learner,Train MSE,Train CV MSE,Coefs
0,LinearRegression,2859.690399,2999.032286,0.651158
1,LarsCV,2932.128955,3849.042608,0.083944
2,Earth,2576.797876,3214.796738,0.225509
3,RandomForestRegressor,626.534669,3432.686859,0.018123
4,Meta (LassoCV),2679.703275,2961.874784,NaN


In [28]:
if np.array([]) or [0]:
    print(1)

1


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """Entry point for launching an IPython kernel.
